In [26]:
ДЗ Кирилюк А.М. Т12О-101М-20

# Реализовать классификатор

Аналогично тому, как посчитали вероятности встретить слово `free` в каждом классе (спам/не спам) 
* в функции `fit()` подсчитать такие вероятности для каждого слова
* в функции `predict()` по формуле байеса (см. лекцию) вычислять вероятность принадлежности входного текста к каждому из классов

Результат предсказания - класс, вероятность принадлежности к которому больше

In [21]:
import pandas as pd
from pprint import pprint
import string
import numpy as np
from typing import List
from copy import deepcopy
from collections import defaultdict

filename = 'data/sms_spam_collection.tar.gz'

df = pd.read_csv(
    filename,
    compression='gzip',
    header=1,
    sep='\t',
    encoding='utf8',
    names=['class', 'sms_text'],
    error_bad_lines=False
)

SPAM_CLASS = 'spam'
NOT_SPAM_CLASS = 'ham'
# последняя строка пустая
df = df[:-1]

num_objects, num_features = df.shape
print(num_objects, num_features)

def text_preprocess(sms_text: str) -> str:
    """Преобразование текста для анализа"""
    text_no_punctuation = ''.join([ char for char in sms_text if char not in string.punctuation ])
    text_lowercase = ' '.join( [ word.lower() for word in text_no_punctuation.split(sep=' ') ])
    
    return text_lowercase



df = df.assign(
    sms_text = df['sms_text'].apply(text_preprocess)
)

#print(df[:4]['sms_text'].values.tolist())

#print(df[:4]['class'].tolist())

"""имплементация наивного байесовского классификатора"""
class NaiveBayes:
    def __init__(self):
        
        self.labels = [NOT_SPAM_CLASS, SPAM_CLASS]
        self.class_labels_proba = None  # априорная вероятность класса, словарь
        self.prior_word_proba = None  # частоты фичей (токенов)
    
    def _set_labels_prior_proba(self, data: list, target: list) -> None:
        """Вычисление априорной вероятности классов
        
        Вызов функции должен инициализировать массив self.class_labels_proba[label]
        
        """
        class_labels_proba = dict.fromkeys(self.labels, 0.0)

        spam_sms_num = len([cls for cls in target if cls == SPAM_CLASS])
        notspam_sms_num = len([cls for cls in target if cls == NOT_SPAM_CLASS])
        print('spam_sms_num',spam_sms_num,'notspam_sms_num',notspam_sms_num)
        
        class_labels_proba[SPAM_CLASS] = spam_sms_num / (spam_sms_num + notspam_sms_num)
        class_labels_proba[NOT_SPAM_CLASS] = notspam_sms_num / (spam_sms_num + notspam_sms_num)

        self.class_labels_proba = class_labels_proba
    
    def _tokenize_text(self, text) -> list:
        """Функция, которая разобьёт входной текст на токены
        
        Токены вернуть в виде списка"""
        
        tokens = text.split()
        
        return tokens
    
    def _set_word_prior_proba(self, data, target):
        """Вычисляем априорную вероятность токенов в классе
        
        Заполняем словарь self.prior_word_proba[label][token]
        
        """
        word_proba_dict_by_class = dict.fromkeys(self.labels)
        word_proba_dict_by_class[SPAM_CLASS] = defaultdict(int)
        word_proba_dict_by_class[NOT_SPAM_CLASS] = defaultdict(int)
        #print(word_proba_dict_by_class)
        

        sum_count = dict.fromkeys(self.labels, 0)
        
        
        for obj, label in zip(data, target):
            #print('{'+obj+'}')
            for token in self._tokenize_text(obj):
                #print('{'+token+'}')
                
                word_proba_dict_by_class[label][token] += 1
                
                
                if not label in sum_count:
                    sum_count[label] = 0
                sum_count[label] += 1
    
        print( word_proba_dict_by_class['ham']['thank'], word_proba_dict_by_class['spam']['thank'] )
        
        for label in word_proba_dict_by_class:
            for token in word_proba_dict_by_class[label]:
                word_proba_dict_by_class[label][token] = word_proba_dict_by_class[label][token] / sum_count[label]
                
        print('tokens=', sum_count['ham']+ sum_count['spam'])
        
        self.prior_word_proba = word_proba_dict_by_class
        

    def fit(self, data: list, target: list):
        """Обучение статистик по датасету

        :param data: массив документов, каждый документ - объект типа str
        :param target: массив меток объектов
        :return:
        """
        if not isinstance(data, list):
            raise ValueError('Аргумент data должен иметь тип list')
        if not isinstance(target, list):
            raise ValueError('Аргумент target должен иметь тип list')
        print('Данные инициализированы!')
        self._set_labels_prior_proba(data, target)
        print(f'Априорные вероятности классов {self.class_labels_proba}')
        self._set_word_prior_proba(data, target)
        print('Обучили априорные вероятности слов')
        

    def _predict_proba(self, data: list) -> List[tuple]:
        """Предсказываем класс для текстовой смс

        :param data: массив документов, для каждого из которых нужно предсказать метку
        :return: вероятности для каждого из классов
        """
        prediction = []
        for obj in data:
            posterior_class_proba = defaultdict(lambda: 1)
            for token in self._tokenize_text(obj):
                for label in self.labels:
                    posterior_class_proba[label] *= self.prior_word_proba[label][token]
            # сохраняем для каждой метки класса - сколько меток, таков и размер uple
            prediction.append(
                tuple(
                    posterior_class_proba[label] for label in self.labels
                )
            )
        print(f'proba: {prediction}')
        return prediction
    
    def predict(self, data) -> List[str]:
        predict_labels = []
        for proba in self._predict_proba(data):
            predict_labels.append(self.labels[np.argmax(proba)])
        return predict_labels

naive_bayes = NaiveBayes()

5570 2


In [22]:
naive_bayes.fit(
    data=df['sms_text'].values.tolist(),
    target=df['class'].tolist()
)

Данные инициализированы!
spam_sms_num 747 notspam_sms_num 4823
Априорные вероятности классов {'ham': 0.8658886894075404, 'spam': 0.1341113105924596}
27 1
tokens= 85640
Обучили априорные вероятности слов


In [23]:
naive_bayes.prior_word_proba['ham']['thank'], naive_bayes.prior_word_proba['spam']['thank']

(0.0003970529845149336, 5.6692556267362094e-05)

Предсказание метки класса

In [24]:
import numpy as np
# рандомный объект датасета

random_obj_ind = np.random.randint(low=0, high=num_objects, size=3)
random_obj_list = df['sms_text'].values[random_obj_ind].tolist()
random_target = df['class'][random_obj_ind].tolist()

print(random_obj_list)
naive_bayes.predict(
    random_obj_list
)

['hi test on  ltgt rd ', 'i had askd u a question some hours before its answer', 'babe  i miiiiiiissssssssss you  i need you  i crave you    geeee  im so sad without you babe  i love you ']
proba: [(9.551858706761021e-16, 0.0), (1.0965540497136704e-31, 0.0), (9.378372803445559e-50, 0.0)]


['ham', 'ham', 'ham']